# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [178]:
# import libraries

import json
import pandas as pd
import re

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

#from sklearn.externals import joblib
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rafaelmata357/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [179]:
# load data from database
def load_data(database_path):
    engine = create_engine('sqlite:///{}'.format(database_path))
    df = pd.read_sql('SELECT * FROM MESSAGES', engine)
    X = df.message 
    y = df.iloc[:,4:]
    
    return X, y

### 2. Write a tokenization function to process your text data

In [180]:
def tokenize(text):
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words('english')]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
       
    
        clean_tokens.append(clean_tok)
  
    return clean_tokens

In [181]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    #print("Labels:", labels)
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [182]:
mclf = MultiOutputClassifier(RandomForestClassifier())

In [183]:
mclf.get_params()

{'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_impurity_split': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': None,
 'estimator__oob_score': False,
 'estimator__random_state': None,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': RandomForestClassifier(),
 'n_jobs': None}

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [184]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])


In [185]:
clf = MultiOutputClassifier(RandomForestClassifier())

In [186]:
filename = 'model.pickle'
pickle.dump(pipeline, open(filename, 'wb'))

In [187]:
loaded_model = pickle.load(open(filename, 'rb'))

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [188]:
database_path = 'salida.db'
X, y = load_data(database_path)

X_train, X_test, y_train, y_test = train_test_split(X, y)

#train classifier
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f9e7a970940>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [189]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [190]:
col = list(y.columns)
print('accuracy {}'.format((y_pred == y_test.values).mean()))

accuracy 0.9478605092733869


In [191]:

for i in range(len(y_test.values[0,:])):
    print('Target: {}'.format(col[i]))
    print('--'*30)
    print(classification_report(y_test.values[:,i], y_pred[:,i]))

Target: related
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.71      0.42      0.53      1565
           1       0.83      0.94      0.88      4937
           2       0.40      0.31      0.35        52

    accuracy                           0.81      6554
   macro avg       0.65      0.56      0.59      6554
weighted avg       0.80      0.81      0.80      6554

Target: request
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5418
           1       0.85      0.49      0.62      1136

    accuracy                           0.90      6554
   macro avg       0.88      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

Target: offer
------------------------------------------------------------
              precision    recall  f1-score   support

        

/Users/rafaelmata357/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6370
           1       0.88      0.08      0.14       184

    accuracy                           0.97      6554
   macro avg       0.92      0.54      0.56      6554
weighted avg       0.97      0.97      0.96      6554

Target: security
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6436
           1       0.00      0.00      0.00       118

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.96      0.98      0.97      6554

Target: military
------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6325
           1       0.77      0.04      0.08       229

    accuracy              

### 6. Improve your model
Use grid search to find better parameters. 

In [192]:
database_path = 'salida.db'
X, y = load_data(database_path)

X_train, X_test, y_train, y_test = train_test_split(X, y)

'''parameters = {'vect__max_df': (0.5, 0.75, 1.0),
              'vect__max_features': (None, 5000, 10000),
              'tfidf__use_idf': (True, False),
              'clf__estimator__n_estimators': [50, 100, 200],
              'clf__estimator__min_samples_split': [2, 3, 4],
               'clf__estimator__criterion': ['gini','entropy']}'''

parameters = {'clf__estimator__n_estimators': [50, 100, 200]
              }

cv = GridSearchCV(pipeline, param_grid=parameters)



In [ ]:
cv.fit(X_train, y_train)

In [ ]:
# predict on test data
y_pred = cv.predict(X_test)
print('accuracy {}'.format((y_pred == y_test.values).mean()))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.